Домашнее задание
ДЗ №3 «Модель на базе техничсекого анализа»

Цель:

Студенты научатся строить простые модели на базе технического ананлиза.

Базовая модель торговой стратегии, которая будет использована в качестве бейзлайна для дальнейшего улучшения.

Цель - получить торговую стратегию, которая ведет себя "лучше" рынка.


Описание/Пошаговая инструкция выполнения домашнего задания:
 - Создать код на Python, который разделит ваши данные на тренировочный, тестовый и валидационный наборы данных.
 - Построить индикаторы тех анализа;
 - Обогатить датасет индикаторами тех анализа;
 - Построить одну или несколько моделей на основе паттернов технического анализа, которая будет принимать торговые решения по бумагам SnP500 и/или криптовалютам.
 - Провести подбор гиперпараметров моделей с использованием обучающей и тестовой выборок.
 - Провести финальное тестирование построенных торговых стратегий на валидационном наборе данных и сравнить их между собой.
 - Сформировать дашборд, показывающий эффективность различных стратегий во времени.

In [39]:
!pip install pandas numpy TA-Lib-Precompiled scikit-learn hyperopt plotly backtesting backtrader
!pip install matplotlib requests_cache
!pip install TA-Lib
!pip install yfinance_cache
!pip install fmp_python
!pip install outlier_utils
!pip install mplfinance
!pip install catboost

  Using cached TA-Lib-Precompiled-0.4.25.tar.gz (276 kB)
  Preparing metadata (setup.py) ... done
  Using cached backtrader-1.9.78.123-py2.py3-none-any.whl.metadata (6.8 kB)
Using cached backtrader-1.9.78.123-py2.py3-none-any.whl (419 kB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for TA-Lib-Precompiled
  Running setup.py clean for TA-Lib-Precompiled
Failed to build TA-Lib-Precompiled
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (TA-Lib-Precompiled)


In [40]:
!pip3 install backtesting
!pip3 install bokeh==3.8.1

In [41]:
import backtesting
from backtesting import Backtest, Strategy

import bokeh
print(bokeh.__version__)
#from backtesting import set_bokeh_output
#set_bokeh_output(notebook=False)


3.8.1


In [42]:
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

In [43]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import matplotlib.pyplot as plt

import yfinance as yf
import talib as ta
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import joblib

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import skew, kurtosis
from statsmodels.tsa.seasonal import STL
import pywt
import mplfinance as mpf
from scipy.fft import fft
from IPython.display import display




In [44]:
#Машинное обучение
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [45]:
class TemaGoldenCrossStrategy(Strategy):
    def init(self):
        # переопределяем Индикатор по колонке Signal
        self.signal = self.I(lambda: self.data.Signal)
        self.previous_signal = 0

    def next(self):
        current_signal = self.signal[-1]

        ########################################################################
        #  previous_signal current_signal   previous_position    current_position
        #          0              1             is_short              close
        #          0              1        != is_long (close)      buy (is_long)
        #
        #         -1              1             is_short              close
        #         -1              1        != is_long (close)      buy (is_long)
        #
        #         (1)             1
        #
        if current_signal != self.previous_signal:
            if current_signal == 1:
                if self.position.is_short:
                    self.position.close()

                if not self.position.is_long:
                    self.buy()

        ########################################################################
        #  previous_signal current_signal   previous_position    current_position
        #          0              -1             is_long              close
        #          0              -1            != is_short            sell
        #
        #          1              -1             is_long              close
        #          1              -1            != is_short            sell
        #
        #        (-1)             -1

            elif current_signal == -1:
                if self.position.is_long:
                    self.position.close()

                if not self.position.is_short:
                    self.sell()

        ########################################################################
        #  previous_signal current_signal   previous_position    current_position
        #          1              0                                  close
        #
        #         -1              0                                  close
        #
        #          0              0                                  close

            elif current_signal == 0:
                if self.position:
                    self.position.close()

        self.previous_signal = current_signal

In [46]:
def visualForBacktestML(train_df):

  good_signals = train_df[train_df['Label'] == 1].copy()
  bad_signals = train_df[train_df['Label'] == 0].copy()

  figure = make_subplots(rows=1, cols=1)

  # График цены закрытия и TEMA
  figure.add_trace(go.Scatter(x=train_df.index, y=train_df['Close'], name='Цена закрытия', line=dict(color='blue')), row=1, col=1)
  figure.add_trace(go.Scatter(x=good_signals.index, y=good_signals['Close'], mode='markers',
                            name='Покупка', marker=dict(symbol='triangle-up', size=10, color='green')), row=1, col=1)
  figure.add_trace(go.Scatter(x=bad_signals.index, y=bad_signals['Close'], mode='markers',
                            name='Продажа', marker=dict(symbol='triangle-down', size=10, color='red')), row=1, col=1)

  # Настройка макета графика
  figure.update_layout(height=800, title='Визуализация прибыльных и убыточных сделок')
  figure.update_yaxes(title_text='Цена', row=1, col=1)

  figure.show()

  return figure

Проверка работы стратегии с использованием формируемой модели машинного обучения и библиотеки backtesting

In [47]:
#Проверка работы стратегии с использованием модели машинного обучения
def backtestWithML(strategy, data, test_data, ml_modelname=None, ml_model=None, model_pred=None):

    #train_df = signals.copy()
    train_signals, train_conditions, datadf = strategy.generate_signals(data)
    train_df = train_signals.copy()

    #train_conditions = conditions.copy()
    #bt_df = signals.copy()

    #print("Разделение на трейн и тест ")
    #split_value = int(len(bt_df) * 0.6)
    #train_df = bt_df.iloc[:split_value]
    ##
    #test_df = bt_df.iloc[:split_value]
    #test_df = bt_df.iloc[split_value:]#.reset_index(drop=True)



    bt_df = train_df.reset_index()
    bt_df.columns = ['Date', 'signal']
    bt_df['shift'] = bt_df['signal'].shift()
    bt_df['shift'] = bt_df['shift'].fillna(0)
    bt_df['action_x'] = bt_df['signal'].ne(bt_df['shift'])

    bt_df['action_id'] = 0
    bt_df['action_id'] = bt_df['signal'].ne(bt_df['signal'].shift()).cumsum()
    bt_df.columns = bt_df.columns.str.capitalize()

    data_backtest = datadf.copy()
    data_backtest.reset_index()
    data_backtest.drop('Signal', axis=1, inplace=True)
    data_merge = pd.merge(bt_df, data_backtest, on='Date', how='inner').dropna()

    data_merge.rename(columns={'Date': 'Datetime'}, inplace=True)
    data_merge["Datetime"] = pd.to_datetime(data_merge["Datetime"])
    data_merge.set_index('Datetime', inplace=True)
    data_merge.columns = data_merge.columns.str.capitalize()

    ##display("Создаем объект класса Backtest")
    # Создаем объект класса Backtest
    ##bt = Backtest(data_merge, TemaMacdStrategy, cash=1_000_000, commission=.002, exclusive_orders=True, finalize_trades=True) # сделки идут последовательно

    ##display("Запускаем бэктест")
    # Запускаем бэктест
    ##stats = bt.run()

    ##db1 = stats[:27]

    # Выводим статистику
    ##display("Выводим статистику")
    ##print(stats[:27])

    # выводим график
    #bt.plot(
    #  plot_equity=True,
    #  plot_drawdown=True,
    #  relative_equity=False,
    #)

    #Успешные и убыточные сделки
    # групируем по id оставляя только первые строки каждой сделки - это наши точки смены сигнала

    data_merge = data_merge.reset_index()
    train_df = data_merge.groupby('Action_id').first().reset_index()
    # добавим столбец с профитом
    train_df['profit'] = 0.0

    # Смещаем столбцы 'close' и 'signal', чтобы получить данные следующего бара
    train_df['next_close'] = train_df['Close'].shift(-1)
    train_df['next_signal'] = train_df['Signal'].shift(-1)

    # Рассчитываем PnL (отчет о прибылях и убытках)
    train_df['profit'] = (train_df['next_close'] - train_df['Close']) * train_df['Signal']

    # Заполняем PnL значением 0 для последнего бара, так как для него нет следующего бара
    train_df['profit'] = train_df['profit'].fillna(0)

    # Добавляем целевую переменную : 0 - cделка принесла убыток, 1 - сделка принесла прибыль
    train_df['Label'] = train_df['profit'].apply(lambda x: 1 if x >= 0 else 0)
    train_df = train_df[train_df['Signal'] != 0]
    train_df.set_index('Datetime', inplace=True)
    #train_df[['Close', 'next_close' , 'Signal', 'next_signal', 'profit', 'Label']]

    # Визуализация
    #visualForBacktestML(train_df)

    ##------------------

    #X_trainbkt, X_testbkt, y_trainbkt, y_testbkt = train_test_split(
    #train_df, train_df['Label'], test_size=0.3, random_state=42, stratify=train_df['Label']
    #)

    display("Создаем объект класса Backtest")

    X_train = train_df[['Signal', 'Open', 'High', 'Low', 'Close', 'Volume', 'Label']]
    # Создаем объект класса Backtest
    bt = Backtest(X_train, TemaGoldenCrossStrategy, cash=1_000_000, commission=.002, exclusive_orders=True, finalize_trades=True) # сделки идут последовательно

    display("Запускаем бэктест")

    # Запускаем бэктест
    stats = bt.run()

    ###bt.plot()

    db1 = stats[:27]

    # Выводим статистику
    #display("Выводим статистику")
    #print(stats[:27])

    ##------------------


    # И оставляем только те строки где мы входили в позицию
    #display("Оставляем только те строки где мы входили в позицию")
    #train_df = train_df[train_df['Signal'] != 0]

    # Удаляем вспомогательные столбцы
    train_df.drop(columns=['next_close', 'next_signal'], inplace=True)

    # Разделяем признаки и метки
    display("Разделяем признаки и метки")
    train_conditions.columns = train_conditions.columns.str.capitalize()
    train_conditions.drop('Signal', axis=1, inplace=True)

    X_train = train_df.filter(train_conditions.columns, axis=1)
    #X_train = train_df[['tema', 'macd', 'macd_signal', 'close', 'signal']]
    y_train = train_df['Label']


    display("RobustScaler")
    #scaler = StandardScaler()
    scaler = RobustScaler()
    x_scaled = scaler.fit_transform(X_train)
    #https://sky.pro/wiki/python/primer-ispolzovaniya-random-forest-classifier/

    # #обучаем модель
    #model = RandomForestClassifier(n_estimators=100, random_state=42)
    # # Определение гиперпараметров для настройки
    #param_grid = {
    #     'n_estimators': [50, 100, 200],
    #     'max_depth': [None, 10, 20, 30],
    #     'min_samples_split': [2, 5, 10],
    #     'min_samples_leaf': [1, 2, 4]
    # }
    # # Создание экземпляра GridSearchCV
    #grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
    # # Поиск лучших параметров
    #grid_search.fit(x_scaled, y_train)
    # # Вывод лучших параметров
    #print(f"Лучшие параметры: {grid_search.best_params_}")
    # # Создание модели с лучшими параметрами
    #model = grid_search.best_estimator_
    #model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth = None, min_samples_leaf = 4, min_samples_split=2)

    #обучаем модель
    #model = RandomForestClassifier(random_state=42)
    model = CatBoostClassifier(random_state=42)
    model.fit(x_scaled, y_train,
              verbose = False,
              plot=False)
    if ml_modelname is not None :
      namemodel = ml_modelname
      model = joblib.load(namemodel)


    # Важность признаков
    feature_importance = pd.Series(model.feature_importances_, index=X_train.columns)
    print("Feature Importance:\n", feature_importance.sort_values(ascending=False))

    # повторяем все тоже самое с данными для теста
    display("Повторяем все тоже самое с данными для теста")

    test_signals, test_conditions, test_df = strategy.generate_signals(test_data)
    visual_signals = test_signals.copy()
    visual_signals = visual_signals.fillna(0)
    visual_signals = visual_signals.reindex(test_data.index, fill_value=0)
    close_prices = test_data['Close'].values
    if test_signals is None or test_signals.empty:
      return None
    test_signals = test_signals.fillna(0)
    test_signals = test_signals.reindex(test_data.index, fill_value=0)
    test_signals = test_signals.reset_index()
    test_signals.columns = ['Date', 'signal']
    test_df.drop('Signal', axis=1, inplace=True)
    test_df = pd.merge(test_signals, test_df, on='Date', how='inner').dropna()
    test_df.rename(columns={'Date': 'Datetime'}, inplace=True)
    test_df["Datetime"] = pd.to_datetime(test_df["Datetime"])
    test_df.set_index('Datetime', inplace=True)
    test_df['shift'] = test_df['signal'].shift()
    test_df['shift'] = test_df['shift'].fillna(0)
    test_df['action_x'] = test_df['signal'].ne(test_df['shift'])
    test_df['action_id'] = 0
    test_df['action_id'] = test_df['signal'].ne(test_df['signal'].shift()).cumsum()
    test_df = test_df.reset_index()
    test_df = test_df.groupby('action_id').first().reset_index()
    test_df['next_close'] = test_df['Close'].shift(-1)
    test_df['next_signal'] = test_df['signal'].shift(-1)
    test_df['profit'] = (test_df['next_close'] - test_df['Close']) * test_df['signal']
    test_df['profit'] = test_df['profit'].fillna(0)
    test_df['Label'] = test_df['profit'].apply(lambda x: 1 if x >= 0 else 0)
    #test_df = test_df[test_df['signal'] != 0]
    test_df.set_index('Datetime', inplace=True)
    test_df.columns = test_df.columns.str.capitalize()


    display("Для предсказания используем данные в столбацах ниже")
    # Для предсказания используем данные в столбацах condition
    #X_test = test_df[['tema', 'macd', 'macd_signal', 'close', 'signal']]
    test_conditions.columns = test_conditions.columns.str.capitalize()
    test_conditions.drop('Signal', axis=1, inplace=True)
    X_test = test_df.filter(test_conditions.columns, axis=1)
    y_test = test_df['Label']

    # делаем предсказание
    y_pred = model.predict(X_test)
    # вариант когда у нас уже есть модель
    if model_pred is not None:
      y_pred = model_pred.copy()

    # описание наших метрик
    # accuracy - точность нашей модели
    # precision - точность предсказания
    # recall - полнота предсказания
    # f1-score - среднее гармоническое точности и полноты
    # support - количество образцов в каждом классе
    # выводим предсказание
    display("Выводим предсказание")
    print(y_pred)
    # выведем нашу точность
    print(accuracy_score(y_test, y_pred))
    # выведем нашу матрицу
    print(confusion_matrix(y_test, y_pred))
    # выведем нашу классификационную отчет
    print(classification_report(y_test, y_pred))

    # Подготовим данные для бэктеста
    test_df['pred'] = y_pred
    test_df['pred'] = test_df['pred'].ffill()
    test_df.columns = test_df.columns.str.capitalize()
    test_df.loc[(test_df['Pred'] == 0) & (test_df['Signal'] != 0), 'Signal'] = 0
    # зануляем сделки предсказанные как убыточные
    #bt_df.loc[(bt_df['pred'] == 0) & (bt_df['Signal'] != 0), 'Signal'] = 0


    bt = Backtest(test_df, TemaGoldenCrossStrategy, cash=1000000, commission=.002, exclusive_orders=True, finalize_trades=True)

    # Запускаем бэктест
    stats = bt.run()

    # Выводим статистику
    print(stats[:27])

    display("Выводим график :")
    # выводим график

    bt.plot(
        plot_equity=True,
        plot_drawdown=True,
        relative_equity=False,
    )

    #backtest_strategy(strategy_params, test_df)

    display("Выводим db1 и db2 :")

    db2 = stats[:27]
    db = pd.concat([db1, db2], axis = 1)
    db.columns = ['db1', 'db2']
    display(db)

   # match = re.search(r"\d+\.\d+", db2.iloc[4])
   # if match:
   #   float_number = np.float(match.group(1))
   #   print(float_number)

    risk_adjusted_return_ml = (
            db2.iloc[7] *
            db2.iloc[12] *
            db2.iloc[13] *
            db2.iloc[7]
        ) / (1 + db2.iloc[17])

    display(f'risk_adjusted_return_ml = {risk_adjusted_return_ml}')

    # Попытка отрисовки временного ряда с помощью не backtesting
    position_open = False
    positions = []
    equity = 1.0
    equity_curve = [equity]

    for i in range(1, len(visual_signals)):
            current_signal = visual_signals.iloc[i]
            current_date = visual_signals.index[i]

            if not position_open and current_signal != 0:
                # Открываем позицию
                position_open = True
                entry_price = close_prices[i]
                entry_signal = current_signal
                entry_date = current_date

            elif position_open and ((current_signal != 0 and current_signal != entry_signal) or i == len(visual_signals)-1):
                # Закрываем позицию
                exit_price = close_prices[i]
                profit = (exit_price - entry_price) / entry_price * entry_signal

                positions.append({
                    'entry_date': entry_date,
                    'exit_date': current_date,
                    'profit': float(profit)
                })
                equity *= (1.0 + profit)
                position_open = False

            equity_curve.append(float(equity))
    # Попытка отрисовки временного ряда с помощью не backtesting

    strategy.positions = positions
    strategy.equity = equity_curve
    strategy.signals = visual_signals

    roc = rocauc(model, X_test, y_test)

    return_ml = {
            'final_equity': db2.iloc[4], #final_equity,
            'n_trades': db2.iloc[21], #n_trades,
            'returns': db2.iloc[7], #returns,
            'max_drawdown': abs(db2.iloc[17]), #max_drawdown,
            'win_rate': db2.iloc[22], #win_rate,
            'sharpe_ratio': db2.iloc[12], #sharpe_ratio,
            'sortino_ratio': db2.iloc[13], #sortino_ratio,
            'profit_factor': db2.iloc[7], #profit_factor
            'rocauc': roc, #rocauc для модели
            'pred': y_pred.copy(), # запоминаем предсказание для набора
            'ml_model': joblib.dump(model, 'random_forest.joblib' + str(roc)),
            'ml_modelname': 'random_forest.joblib' + str(roc)
        }

    return return_ml


In [48]:
# Класс для базовой стратегии
class BaseStrategy:
    def __init__(self):
        self.positions = []
        self.equity = []

    def calculate_metrics(self, data):
        """Описание используемых метрик производительности:
           final_equity: Конечный капитал после завершения торгового периода. Показывает, насколько вырос или уменьшился начальный капитал в результате торговли.
           Используется для оценки общей эффективности стратегии. Чем выше конечный капитал, тем успешнее стратегия.
           n_trades: Количество сделок, совершенных за период тестирования. Показывает активность стратегии. Большое количество сделок может указывать на высокую частоту торговли, что может быть как преимуществом, так и недостатком в зависимости от контекста.
           returns: Общая доходность стратегии в процентах за период тестирования. Основной показатель прибыльности стратегии. Положительная доходность указывает на успешность стратегии, отрицательная — на убыточность.
           max_drawdown: Максимальная просадка капитала за период тестирования. Показывает наибольшее падение капитала от пика до минимума. Используется для оценки риска стратегии. Чем меньше просадка, тем стабильнее стратегия.
           win_rate: Процент прибыльных сделок от общего количества сделок. Показывает, насколько часто стратегия приносит прибыль. Высокий процент выигрышных сделок может указывать на стабильность стратегии.
           sharpe_ratio: Коэффициент Шарпа измеряет доходность с поправкой на риск. Чем выше значение, тем лучше стратегия с точки зрения доходности на единицу риска. Используется для сравнения стратегий с учетом риска. Высокий коэффициент Шарпа указывает на эффективное использование риска.
           sortino_ratio: Коэффициент Сортино, который измеряет доходность с поправкой на риск, учитывая только негативную волатильность. Аналогичен коэффициенту Шарпа, но фокусируется на риске убытков, что делает его более подходящим для оценки стратегий с асимметричным распределением доходности.
           profit_factor: Отношение общей прибыли к общему убытку. Показывает, насколько прибыльна стратегия по сравнению с убытками. Используется для оценки соотношения прибыли и убытков. Значение больше 1 указывает на прибыльность стратегии.
        """
        if not self.positions:
            return {
                'final_equity': 1.0,
                'n_trades': 0,
                'returns': 0,
                'max_drawdown': 0,
                'win_rate': 0,
                'sharpe_ratio': 0,
                'sortino_ratio': 0,
                'profit_factor': 0
            }

        # Расчет базовых метрик
        final_equity = self.equity[-1]
        n_trades = len(self.positions)
        returns = (final_equity - 1.0) * 100

        # Расчет максимальной просадки
        peak = 1.0
        drawdowns = []
        for eq in self.equity:
            if eq > peak:
                peak = eq
            drawdown = (peak - eq) / peak * 100
            drawdowns.append(drawdown)
        max_drawdown = max(drawdowns)

        # Расчет win rate
        profitable_trades = sum(1 for pos in self.positions if pos['profit'] > 0)
        win_rate = profitable_trades / n_trades * 100 if n_trades > 0 else 0

        # Расчет Sharpe и Sortino ratio
        daily_returns = pd.Series(self.equity).pct_change().dropna()
        if len(daily_returns) > 0:
            sharpe_ratio = np.sqrt(252) * daily_returns.mean() / daily_returns.std()
            downside_returns = daily_returns[daily_returns < 0]
            sortino_ratio = np.sqrt(252) * daily_returns.mean() / downside_returns.std() if len(downside_returns) > 0 else 0
        else:
            sharpe_ratio = sortino_ratio = 0

        # Расчет profit factor
        gross_profit = sum(pos['profit'] for pos in self.positions if pos['profit'] > 0)
        gross_loss = abs(sum(pos['profit'] for pos in self.positions if pos['profit'] < 0))
        profit_factor = gross_profit / gross_loss if gross_loss != 0 else 0

        return {
            'final_equity': final_equity,
            'n_trades': n_trades,
            'returns': returns,
            'max_drawdown': max_drawdown,
            'win_rate': win_rate,
            'sharpe_ratio': sharpe_ratio,
            'sortino_ratio': sortino_ratio,
            'profit_factor': profit_factor
        }

In [49]:
# Стратегия 4: Золотой крест
#https://sergeytereshkin.ru/library/zolotoy-krest-signal-razvorota-i-metodika-poiska-na-grafike
class Strategy4(BaseStrategy):
    """
    «Золотой крест» — один из самых надежных бычьих сигналов технического анализа,
    позволяющий трейдерам структурировать вход в длинную позицию при строгом риск-менеджменте и подтверждении индикаторами.

    Выбор и настройка скользящих средних
    Оптимальные периоды для SMA
     - Для дневных и недельных графиков оптимальным сочетанием является SMA50/SMA200, поскольку SMA сглаживает шум и отражает среднесрочную и долгосрочную динамику цены.

    Использование EMA на коротких таймфреймах
     - На коротких таймфреймах (M15–H1) предпочтительнее EMA50/EMA200: EMA быстрее реагирует на последние изменения цены, что позволяет оперативнее фиксировать разворот, но требует дополнительной фильтрации шума.

    Бэктестинг настроек MA
     - Оптимальные параметры следует подбирать через бэктестинг: тестировать разные комбинации периодов,
       фиксируя доходность и максимальную просадку каждой стратегии.

    Подтверждающие индикаторы и фильтры
     - MACD как индикатор тренда
     - MACD подтверждает силу бычьего сигнала при росте гистограммы выше нулевой линии и пересечении сигнальной линии снизу вверх,
       что усиливает вероятность продолжения восходящего движения.

    RSI и оценка перекупленности
     - RSI в диапазоне 50–70 указывает на адекватный бычий импульс без перегрева рынка и помогает отсеивать ложные сигналы «золотого креста» во время флета.

    Анализ объемов и свечных паттернов
     - Дополнительным фильтром служат объемы: рост торгового объема в момент пересечения показывает активное участие игроков и подтверждает значимость сигнала.
       Бычьи свечные формации, такие как поглощение или «утренняя звезда», рядом с точкой пересечения усиливают доверие к сигналу.

    Источник Yandex:
    Некоторые оптимальные параметры стратегии Golden Cross в трейдинге:
      - Первичные условия: пересечение MA(50) над MA(200) с закреплением выше на 2+ торговых дня.
      - Направленность скользящих средних: обе MA направлены вверх под углом не менее 5°.
      - Расстояние между MA: минимальный разрыв 1,5% для исключения случайных пересечений.
      - Объёмное подтверждение: превышение 20-дневного среднего объёма на 30%+.
      - Дополнительные фильтры качества:
         RSI(14) в диапазоне 45–70 (для исключения экстремальных состояний перекупленности),
         MACD выше нулевой линии (для подтверждения смены импульса на бычий),
         отсутствие сильных уровней сопротивления в ближайших 8–12% от цены пересечения,
         макроэкономический фон (отсутствие ожидаемых негативных событий в ближайшие 4–6 недель).

    """
    def __init__(self, bb_period=20, bb_std=2, rsi_period=14, sma=10, ema_period=4, ma_period=10, macd_fast=12, macd_slow=26, macd_signal=9):
        super().__init__()
        self.bb_period = int(bb_period)
        self.bb_std = float(bb_std)
        self.rsi_period = int(rsi_period)
        self.sma = int(sma)
        self.ema_period = int(ema_period)
        self.ma_period = int(ma_period)
        self.macd_fast = int(macd_fast)
        self.macd_slow = int(macd_slow)
        self.macd_signal = int(macd_signal)

    def get_params(self):
        """Возвращает текущие гиперпараметры стратегии"""
        return {
            'bb_period': self.bb_period,
            'bb_std': self.bb_std,
            'rsi_period': self.rsi_period,
            'sma': self.sma,
            'ema_period': self.ema_period,
            'ma_period': self.ma_period,
            'macd_fast': self.macd_fast,
            'macd_slow': self.macd_slow,
            'macd_signal': self.macd_signal
        }

    def generate_signals(self, data):
        try:
            df = data.copy()

            close_prices = df['Close'].values.astype(float).flatten()
            open_prices = df['Open'].values.astype(float).flatten()
            high_prices = df['High'].values.astype(float).flatten()
            low_prices = df['Low'].values.astype(float).flatten()
            volume = df['Volume'].values.astype(float).flatten()

            #Полосы Боллинджера - Уровни поддержки и сопротивления
            df['BB_middle'] = ta.SMA(close_prices, timeperiod=self.bb_period)
            std_dev = ta.STDDEV(close_prices, timeperiod=self.bb_period)
            df['BB_upper'] = df['BB_middle'] + self.bb_std * std_dev
            df['BB_lower'] = df['BB_middle'] - self.bb_std * std_dev

            df['sma'] = ta.SMA(close_prices, timeperiod=self.sma)
            df['ma_period'] = ta.MA(close_prices, timeperiod=self.ma_period)
            df['RSI'] = ta.RSI(close_prices, timeperiod=self.rsi_period)
            #df['obv'] = np.where(df['Close'] > df['Close'].shift(1), df['Volume'], np.where(df['Close'] < df['Close'].shift(1), -df['Volume'], 0)).cumsum()
            df['obv'] = ta.OBV(close_prices, volume)
            df['engulfing'] = ta.CDLENGULFING(open_prices, high_prices, low_prices, close_prices)
            df['morningstar'] = ta.CDLMORNINGSTAR(open_prices, high_prices, low_prices, close_prices)
            #df['adosc'] = ta.ADOSC(high_prices, low_prices, close_prices, volume)

            macd, signal, _ = ta.MACD(close_prices,
                              fastperiod=self.macd_fast,
                              slowperiod=self.macd_slow,
                              signalperiod=self.macd_signal)
            df['MACD'] = macd
            df['Signal'] = signal
            #df = df.fillna(method='ffill').fillna(method='bfill')
            df = df.ffill().copy()
            signals = pd.Series(index=df.index, data=0)

            long_condition = ((df['engulfing'] > 0) | (df['morningstar'] > 0)) #& ((df['Close'] < df['sma']) & (df['Close'].shift(1) >= df['sma'].shift(1))) & (df['MACD'] > df['Signal']) & (df['RSI'] > 50) & (df['RSI'] < 70) & (df['obv'] > 0)
            #long_condition =  (df['RSI'] > 50) & (df['sma_period'] > 50) & (df['Close'] > df['EMA']) & (df['MACD'] > df['Signal']) & (df['obv'] > 0)
            #short_condition =  (df['RSI'] < 70) & (df['sma_period'] < 200) & (df['Close'] < df['EMA']) & (df['MACD'] < df['Signal']) & (df['obv'] < 0)
            #short_condition = Стоп-лосс | тейк-профит | трейлинг-стоп
            short_condition =  (((df['sma'] < 50)) | ((df['Close'] < df['sma']) & (df['Close'].shift(1) >= df['sma'].shift(1)))) | ((df['RSI'] > 50) & (df['MACD'] < df['Signal'])) | (df['Close'] > df['BB_upper'])
            signals[long_condition] = 1
            signals[short_condition] = -1
            signals = signals.reindex_like(data)


            conditions = df.copy()
            conditions = conditions.reset_index()
            conditions = df.filter(['BB_middle', 'BB_upper',
                                    'BB_lower', 'sma', 'ma_period', 'RSI', 'obv', 'engulfing',
                                    'morningstar', 'MACD', 'Signal'], axis=1)
            conditions = conditions.dropna()

            return signals, conditions, df
        except Exception as e:
            print(f"Ошибка в generate_signals для Strategy4: {str(e)}")
            return None

In [50]:
# Стратегия 3: RSI + Candlestick Patterns
class Strategy3(BaseStrategy):
    """
    Комбинированная стратегия разворота тренда, использующая:
    - RSI (Relative Strength Index) - для определения перекупленности/перепроданности
    - Паттерны свечей - для подтверждения разворота (Doji, Hammer, Engulfing)

    Бизнес-логика стратегии:
    1. Основной принцип:
       - Поиск точек разворота в перекупленных/перепроданных зонах
       - Использование паттернов свечей для подтверждения разворота

    2. Условия для входа в длинную позицию (Long):
       - RSI находится в зоне перепроданности (< 30)
       - Формируется один из бычьих паттернов:
         * Doji (свеча неопределенности)
         * Hammer (молот)
         * Bullish Engulfing (бычье поглощение)

    3. Условия для входа в короткую позицию (Short):
       - RSI находится в зоне перекупленности (> 70)
       - Формируется один из медвежьих паттернов:
         * Doji (свеча неопределенности)
         * Inverted Hammer (перевернутый молот)
         * Bearish Engulfing (медвежье поглощение)

    4. Управление рисками:
       - RSI фильтрует слабые сигналы
       - Требуется подтверждение паттернами свечей
       - Учитывается сила сигнала разворота

    Особенности настройки:
    - rsi_period: период RSI (по умолчанию 14)
    - Паттерны свечей имеют фиксированные правила идентификации

    Преимущества стратегии:
    - Сочетание технического индикатора и визуального анализа
    - Четкие правила входа и выхода
    - Возможность раннего обнаружения разворотов тренда
    """
    def __init__(self, rsi_period=14):
        super().__init__()
        self.rsi_period = int(rsi_period)

    def get_params(self):
        """Возвращает текущие гиперпараметры стратегии"""
        return {
            'rsi_period': self.rsi_period
        }

    def generate_signals(self, data):
        try:
            df = data.copy()
            close_prices = df['Close'].values.astype(float).flatten()
            open_prices = df['Open'].values.astype(float).flatten()
            high_prices = df['High'].values.astype(float).flatten()
            low_prices = df['Low'].values.astype(float).flatten()
            df['RSI'] = ta.RSI(close_prices, timeperiod=self.rsi_period)
            df['doji'] = ta.CDLDOJI(open_prices, high_prices, low_prices, close_prices)
            df['hammer'] = ta.CDLHAMMER(open_prices, high_prices, low_prices, close_prices)
            df['engulfing'] = ta.CDLENGULFING(open_prices, high_prices, low_prices, close_prices)
            df = df.ffill().copy()
            #df = df.fillna(method='ffill').fillna(method='bfill')
            signals = pd.Series(index=df.index, data=0)
            long_condition = ((df['doji'] > 0) | (df['hammer'] > 0) | (df['engulfing'] > 0)) & (df['RSI'] < 30)
            short_condition = ((df['doji'] < 0) | (df['hammer'] < 0) | (df['engulfing'] < 0)) & (df['RSI'] > 70)
            signals[long_condition] = 1
            signals[short_condition] = -1
            signals = signals.reindex_like(data)

            conditions = df.copy()
            conditions = conditions.reset_index()
            conditions = df.filter(['RSI', 'doji',
                                    'hammer', 'engulfing'], axis=1)
            conditions = conditions.dropna()

            return signals, conditions, df
        except Exception as e:
            print(f"Ошибка в generate_signals для Strategy3: {str(e)}")
            return None

In [51]:
class Strategy2(BaseStrategy):
    """
    Стратегия торговли на отскок (Mean Reversion), использующая:
    - Bollinger Bands - для определения волатильности и экстремальных значений цены
    - Stochastic Oscillator - для подтверждения разворота

    Бизнес-логика стратегии:
    1. Основной принцип:
       - Торговля от краев полос Боллинджера к средней линии
       - Использование стохастика для подтверждения сигналов разворота

    2. Условия для входа в длинную позицию (Long):
       - Цена находится ниже нижней полосы Боллинджера (перепроданность)
       - Линия %K стохастика пересекает линию %D снизу вверх (подтверждение разворота вверх)

    3. Условия для входа в короткую позицию (Short):
       - Цена находится выше верхней полосы Боллинджера (перекупленность)
       - Линия %K стохастика пересекает линию %D сверху вниз (подтверждение разворота вниз)

    4. Управление рисками:
       - Полосы Боллинджера автоматически адаптируются к волатильности
       - Стохастик подтверждает потенциал разворота
       - Торговля против тренда только при сильных сигналах

    Особенности настройки:
    - bb_period: период Bollinger Bands (по умолчанию 20)
    - bb_std: количество стандартных отклонений (по умолчанию 2)
    - stoch_k/d/slow: параметры стохастика, влияющие на его чувствительность
    """
    def __init__(self, bb_period=20, bb_std=2, stoch_k=14, stoch_d=3, stoch_slow=3, rsi_period = 14):
        super().__init__()
        self.bb_period = int(bb_period)
        self.bb_std = float(bb_std)
        self.stoch_k = int(stoch_k)
        self.stoch_d = int(stoch_d)
        self.stoch_slow = int(stoch_slow)
        self.rsi_period = int(rsi_period)

    def get_params(self):
        """Возвращает текущие гиперпараметры стратегии"""
        return {
            'bb_period': self.bb_period,
            'bb_std': self.bb_std,
            'stoch_k': self.stoch_k,
            'stoch_d': self.stoch_d,
            'stoch_slow': self.stoch_slow,
            'rsi_period': self.rsi_period
        }

    def generate_signals(self, data):
        try:
            df = data.copy()
            close_prices = df['Close'].values.astype(float).flatten()
            high_prices = df['High'].values.astype(float).flatten()
            low_prices = df['Low'].values.astype(float).flatten()
            df['BB_middle'] = ta.SMA(close_prices, timeperiod=self.bb_period)
            std_dev = ta.STDDEV(close_prices, timeperiod=self.bb_period)
            df['BB_upper'] = df['BB_middle'] + self.bb_std * std_dev
            df['BB_lower'] = df['BB_middle'] - self.bb_std * std_dev
            df['RSI'] = ta.RSI(close_prices, timeperiod=self.rsi_period)
            slowk, slowd = ta.STOCH(high_prices,
                                   low_prices,
                                   close_prices,
                                   fastk_period=self.stoch_k,
                                   slowk_period=self.stoch_d,
                                   slowd_period=self.stoch_slow)
            df['slowk'] = slowk
            df['slowd'] = slowd
            df = df.ffill().copy()
            #df = df.fillna(method='ffill').fillna(method='bfill')
            signals = pd.Series(index=df.index, data=0)
            long_condition = (df['Close'] < df['BB_lower']) & (df['slowk'] > df['slowd']) & (df['RSI'] < 70)
            short_condition = (df['Close'] > df['BB_upper']) & (df['slowk'] < df['slowd']) & (df['RSI'] > 30)
            signals[long_condition] = 1
            signals[short_condition] = -1
            signals = signals.reindex_like(data)

            conditions = df.copy()
            conditions = conditions.reset_index()
            conditions = df.filter(['BB_middle', 'BB_upper',
                                    'BB_lower', 'RSI', 'slowk', 'slowd', 'Close'], axis=1)
            conditions = conditions.dropna()

            return signals, conditions, df
        except Exception as e:
            print(f"Ошибка в generate_signals для Strategy2: {str(e)}")
            return None




In [52]:
class Strategy1(BaseStrategy):
    """
    Комбинированная трендово-импульсная стратегия, использующая:
    - EMA (Exponential Moving Average) - для определения тренда
    - RSI (Relative Strength Index) - для определения перекупленности/перепроданности
    - MACD (Moving Average Convergence Divergence) - для определения моментума и точек входа/выхода

    Бизнес-логика стратегии:
    1. Определение тренда:
       - Цена выше EMA = восходящий тренд
       - Цена ниже EMA = нисходящий тренд

    2. Условия для входа в длинную позицию (Long):
       - Цена находится выше EMA (подтверждение восходящего тренда)
       - RSI < 70 (нет перекупленности)
       - MACD пересекает Signal Line снизу вверх (сигнал усиления восходящего импульса)

    3. Условия для входа в короткую позицию (Short):
       - Цена находится ниже EMA (подтверждение нисходящего тренда)
       - RSI > 30 (нет перепроданности)
       - MACD пересекает Signal Line сверху вниз (сигнал усиления нисходящего импульса)

    4. Управление рисками:
       - RSI используется как фильтр для предотвращения входа в перекупленной/перепроданной зоне
       - EMA подтверждает направление тренда
       - MACD подтверждает силу импульса

    Особенности настройки:
    - ema_period: период EMA (по умолчанию 20) - чем больше период, тем более долгосрочный тренд отслеживается
    - rsi_period: период RSI (по умолчанию 14) - влияет на чувствительность индикатора к изменениям цены
    - macd_fast/slow/signal: периоды MACD - влияют на скорость реакции на изменение тренда
    """
    def __init__(self, ema_period=20, rsi_period=14, macd_fast=12, macd_slow=26, macd_signal=9):
        super().__init__()
        self.ema_period = int(ema_period)
        self.rsi_period = int(rsi_period)
        self.macd_fast = int(macd_fast)
        self.macd_slow = int(macd_slow)
        self.macd_signal = int(macd_signal)

    def get_params(self):
        """Возвращает текущие гиперпараметры стратегии"""
        return {
            'ema_period': self.ema_period,
            'rsi_period': self.rsi_period,
            'macd_fast': self.macd_fast,
            'macd_slow': self.macd_slow,
            'macd_signal': self.macd_signal
        }

    def generate_signals(self, data):
        try:
            df = data.copy()
            close_prices = df['Close'].values.astype(float).flatten()
            df['EMA'] = ta.EMA(close_prices, timeperiod=self.ema_period)
            df['RSI'] = ta.RSI(close_prices, timeperiod=self.rsi_period)
            macd, signal, _ = ta.MACD(close_prices,
                                      fastperiod=self.macd_fast,
                                      slowperiod=self.macd_slow,
                                      signalperiod=self.macd_signal)
            df['MACD'] = macd
            df['Signal'] = signal
            df = df.ffill().copy()
            #df = df.fillna(method='ffill').fillna(method='bfill')
            signals = pd.Series(index=df.index, data=0)
            long_condition = (df['Close'] > df['EMA']) & (df['RSI'] < 70) & (df['MACD'] > df['Signal'])
            short_condition = (df['Close'] < df['EMA']) & (df['RSI'] > 30) & (df['MACD'] < df['Signal'])
            signals[long_condition] = 1
            signals[short_condition] = -1
            signals = signals.reindex_like(data)

            conditions = df.copy()
            conditions = conditions.reset_index()
            conditions = df.filter(['EMA', 'RSI',
                                    'MACD', 'Signal', 'Close'], axis=1)
            conditions = conditions.dropna()

            return signals, conditions, df
        except Exception as e:
            print(f"Ошибка в generate_signals для Strategy1: {str(e)}")
            return None


In [53]:
# Определение пространств параметров для оптимизации
param_spaces = {
    'Strategy1': {
        'ema_period': hp.quniform('ema_period', 10, 50, 1),
        'rsi_period': hp.quniform('rsi_period', 7, 21, 1),
        'macd_fast': hp.quniform('macd_fast', 8, 20, 1),
        'macd_slow': hp.quniform('macd_slow', 21, 40, 1),
        'macd_signal': hp.quniform('macd_signal', 5, 15, 1)
    },
    'Strategy2': {
        'bb_period': hp.quniform('bb_period', 10, 50, 1),
        'bb_std': hp.uniform('bb_std', 1.5, 3.0),
        'stoch_k': hp.quniform('stoch_k', 7, 21, 1),
        'stoch_d': hp.quniform('stoch_d', 3, 7, 1),
        'stoch_slow': hp.quniform('stoch_slow', 3, 7, 1),
        'rsi_period': hp.quniform('rsi_period', 7, 21, 1)
    },
    'Strategy3': {
        'rsi_period': hp.quniform('rsi_period', 7, 21, 1)
    },
    'Strategy4': {
        'bb_period': hp.quniform('bb_period', 10, 50, 1),
        'bb_std': hp.uniform('bb_std', 1.5, 3.0),
        'rsi_period': hp.quniform('rsi_period', 7, 21, 1),
        'sma': hp.quniform('sma', 10, 50, 1),
        'ema_period': hp.quniform('ema_period', 10, 50, 1),
        'ma_period': hp.quniform('ma_period', 10, 50, 1),
        'macd_fast': hp.quniform('macd_fast', 8, 20, 1),
        'macd_slow': hp.quniform('macd_slow', 21, 40, 1),
        'macd_signal': hp.quniform('macd_signal', 5, 15, 1)
    }
}

In [54]:
# Функция для бэктестирования Buy & Hold стратегии
def backtest_buy_hold(data):
    """Бэктестирование Buy & Hold стратегии"""
    initial_price = float(data['Close'].iloc[0])
    final_price = float(data['Close'].iloc[-1])
    returns = (final_price - initial_price) / initial_price * 100

    daily_returns = data['Close'].pct_change().dropna()

    if len(daily_returns) > 0:
        sharpe_ratio = np.sqrt(252) * daily_returns.mean() / daily_returns.std()
        downside_returns = daily_returns[daily_returns < 0]
        sortino_ratio = np.sqrt(252) * daily_returns.mean() / downside_returns.std() if len(downside_returns) > 0 else 0
    else:
        sharpe_ratio = sortino_ratio = 0

    rolling_max = data['Close'].expanding().max()
    drawdowns = (rolling_max - data['Close']) / rolling_max
    max_drawdown = float(drawdowns.max() * 100)

    return {
        'final_equity': 1 + returns / 100,
        'n_trades': 1,
        'returns': returns,
        'max_drawdown': max_drawdown,
        'win_rate': 100.0 if returns > 0 else 0.0,
        'sharpe_ratio': float(sharpe_ratio),
        'sortino_ratio': float(sortino_ratio),
        'profit_factor': float('inf') if returns > 0 else 0.0
    }


In [55]:
def optimize_strategy(strategy_class, data, param_space):
    def objective(params):
        strategy = strategy_class(**params)
        metrics, conditions, signals, df = backtest_strategy(strategy, data)
        if metrics:
            return {'loss': -metrics['profit_factor'], 'status': STATUS_OK}
        else:
            return {'loss': 0, 'status': STATUS_OK}

    trials = Trials()
    best = fmin(fn=objective,
                space=param_space,
                algo=tpe.suggest,
                max_evals=50,
                trials=trials)

    return best

In [56]:
def format_metric(value):
    """Форматирует число с двумя знаками после точки, если есть дробная часть"""
    if isinstance(value, (int, float)):
        return round(value, 2) if not value.is_integer() else int(value)
    return value

In [57]:
# Функция для создания сводной таблицы результатов
def create_summary_table(results, instrument):
    """Создание сводной таблицы результатов"""
    metrics = ['final_equity', 'n_trades', 'returns', 'max_drawdown',
               'win_rate', 'sharpe_ratio', 'sortino_ratio', 'profit_factor']

    # Создаем MultiIndex для более структурированного отображения
    index_tuples = []
    data = []

    # Buy & Hold результаты
    if 'Buy & Hold' in results and 'full' in results['Buy & Hold']:
        bh_metrics = results['Buy & Hold']['full']
        index_tuples.append(('Buy & Hold', 'Full'))
        data.append({
            'Days': len(results['Buy & Hold']['full_data']),
            'Hyperparameters': 'N/A',
            **{metric: format_metric(bh_metrics[metric]) for metric in metrics}
        })

    # Результаты торговых стратегий
    #for strategy in ['Strategy2', 'Strategy3', 'Strategy1', 'Strategy4']:
    for strategy in ['Strategy4']:
        if strategy in results:
            for dataset in ['train_default', 'test', 'valid', 'test_ml']:
                if dataset in results[strategy]:
                    metrics_data = results[strategy][dataset]

                    # Определяем, какие гиперпараметры использовать
                    if dataset == 'train_default':
                        hyperparams = results[strategy].get('train_default_params', 'N/A')
                    elif dataset == 'test':
                        hyperparams = results[strategy].get('optimized_params', 'N/A')
                    elif dataset == 'test_ml':
                        hyperparams = results[strategy].get('optimized_params_ml', 'N/A')
                    elif dataset == 'valid':
                        # Для Valid используем гиперпараметры, оптимизированные на объединенном наборе Train и Test
                        hyperparams = results[strategy].get('optimized_params_combined', 'N/A')

                    days = len(results[strategy][f'{dataset}_data'])

                    # Заменяем "train_default" на "Train"
                    dataset_name = 'Train' if dataset == 'train_default' else dataset.capitalize()

                    # Форматируем гиперпараметры с двумя знаками после точки, если есть дробная часть
                    formatted_hyperparams = {}
                    if hyperparams != 'N/A':
                        for param, value in hyperparams.items():
                            formatted_hyperparams[param] = round(value, 2) if isinstance(value, float) else value
                    else:
                        formatted_hyperparams = 'N/A'

                    index_tuples.append((strategy, dataset_name))
                    data.append({
                        'Days': days,
                        'Hyperparameters': formatted_hyperparams,
                        **{metric: format_metric(metrics_data[metric]) for metric in metrics}
                    })

    # Создаем DataFrame с MultiIndex
    df = pd.DataFrame(data, index=pd.MultiIndex.from_tuples(index_tuples, names=['Strategy', 'Dataset']))

    # Форматируем таблицу для красивого отображения
    styled_df = df.style\
        .format({
            'final_equity': '{:.2f}',
            'returns': '{:.2f}%',
            'max_drawdown': '{:.2f}%',
            'win_rate': '{:.2f}%',
            'sharpe_ratio': '{:.2f}',
            'sortino_ratio': '{:.2f}',
            'profit_factor': '{:.2f}'
        })\
        .set_caption(f'Сводная таблица результатов бэктестирования стратегий для {instrument}')\
        .set_table_styles([
            {'selector': 'caption', 'props': [('text-align', 'center'), ('font-size', '16px'), ('font-weight', 'bold')]},
            {'selector': 'th', 'props': [('text-align', 'center'), ('background-color', '#f0f0f0')]},
            {'selector': 'td', 'props': [('text-align', 'right')]}
        ])\
        .background_gradient(subset=['profit_factor'], cmap='RdYlGn')\
        .background_gradient(subset=['returns'], cmap='RdYlGn')

    return styled_df

In [58]:
def analyze_trades(strategy, data, signals):
    """ Анализ фактических сделок на основе сигналов и ценовых данных. Возвращает список сделок с точками входа/выхода и прибылью. """
    trades = []
    in_position = False
    entry_price = 0
    entry_date = None
    position_type = None  # 'long' or 'short'
    for i in range(len(data)):
        current_date = data.index[i]
        current_price = data['Close'].iloc[i]
        is_dataframe = isinstance(signals, pd.DataFrame)
        if is_dataframe:
          current_signal = signals['Signal'].iloc[i]
        elif not is_dataframe:
          current_signal = signals.iloc[i]
        if not in_position and current_signal != 0:
            # Opening a position
            in_position = True
            entry_price = current_price
            entry_date = current_date
            position_type = 'long' if current_signal == 1 else 'short'
        elif in_position:
            # Check for closing conditions
            should_close = (
                (position_type == 'long' and current_signal == -1) or
                (position_type == 'short' and current_signal == 1) or
                (i == len(data) - 1)  # Close at the end of the period
            )
            if should_close:
                # Calculate profit
                profit = (
                    (current_price - entry_price) / entry_price * 100
                    if position_type == 'long'
                    else (entry_price - current_price) / entry_price * 100
                )
                trades.append({
                    'entry_date': entry_date,
                    'entry_price': entry_price,
                    'exit_date': current_date,
                    'exit_price': current_price,
                    'type': position_type,
                    'profit': profit
                })
                in_position = False
                entry_price = 0
                entry_date = None
                position_type = None
    return trades

In [59]:
# Функция оценки стратегических показателей и расчета доходности с поправкой на риск
def calculate_risk_adjusted_returns(metrics):
  #https://www.wallstreetprep.com/knowledge/risk-adjusted-return/
    """Risk-Adjusted Returns позволяет выбрать стратегию, которая приносит доходность с минимальным риском..
       Он позволяет сравнивать стратегии не только по их абсолютной доходности, но и по тому, насколько эффективно они используют риск для достижения этой доходности.
       Оценить, насколько стратегия эффективно использует риск для достижения доходности. Это позволяет сравнивать стратегии, которые могут иметь схожую доходность, но разный уровень риска.
       Компоненты формулы: returns: Общая доходность стратегии. Чем выше доходность, тем выше значение метрики.
                           sharpe_ratio: Коэффициент Шарпа, который измеряет доходность с поправкой на общий риск (волатильность). Чем выше коэффициент Шарпа, тем лучше стратегия использует риск для получения доходности.
                           sortino_ratio: Коэффициент Сортино, который измеряет доходность с поправкой на риск убытков (негативную волатильность). Чем выше коэффициент Сортино, тем лучше стратегия управляет риском убытков.
                           profit_factor: Отношение общей прибыли к общему убытку. Чем выше значение, тем больше прибыль по сравнению с убытками.
                           max_drawdown: Максимальная просадка капитала. Чем меньше просадка, тем стабильнее стратегия. В формуле используется 1 + max_drawdown, чтобы избежать деления на ноль и уменьшить влияние больших просадок.
        Логика расчета: В числителе стоит произведение доходности, коэффициента Шарпа, коэффициента Сортино и фактора прибыли. Это позволяет учитывать как абсолютную доходность, так и эффективность управления риском.
                        В знаменателе - метрика максимальной просадки, что снижает значение метрики, если стратегия имеет высокий уровень риска (большие просадки).
    """
    if not metrics:
        return float('-inf')

    # Нормализуем доходность по показателям риска
    risk_adjusted_return = (
        metrics['returns'] *
        metrics['sharpe_ratio'] *
        metrics['sortino_ratio']
        #metrics['profit_factor']
    ) / (1 + metrics['max_drawdown'])  # Чтобы избежать деления на ноль.

    return risk_adjusted_return

In [60]:

def plot_best_strategy(data, strategy, signals, equity_curve, strategy_results, trades, title):
    """ Функция визуализации, показывающая реальные сделки стратегии вместо сигналов """
    fig = make_subplots(rows=2, cols=1,
                        shared_xaxes=True,
                        vertical_spacing=0.03,
                        row_heights=[0.7, 0.3],
                        subplot_titles=(
                            "Price and Trades",  # Убираем дублирование заголовка
                            "Equity Curve"
                        ))

    # Candlestick chart
    fig.add_trace(
        go.Candlestick(
            x=data.index,
            open=data['Open'],
            high=data['High'],
            low=data['Low'],
            close=data['Close'],
            name='Price'
        ),
        row=1, col=1
    )

    # Отображение сделок
    long_entries = []
    long_exits = []
    short_entries = []
    short_exits = []

    for trade in trades:
        if trade['type'] == 'long':
            long_entries.append({
                'date': trade['entry_date'],
                'price': trade['entry_price'],
                'profit': trade['profit']
            })
            long_exits.append({
                'date': trade['exit_date'],
                'price': trade['exit_price'],
                'profit': trade['profit']
            })
        else:  # short trade
            short_entries.append({
                'date': trade['entry_date'],
                'price': trade['entry_price'],
                'profit': trade['profit']
            })
            short_exits.append({
                'date': trade['exit_date'],
                'price': trade['exit_price'],
                'profit': trade['profit']
            })

    # Отображение сделок long
    if long_entries:
        fig.add_trace(
            go.Scatter(
                x=[t['date'] for t in long_entries],
                y=[t['price'] * 0.99 for t in long_entries],
                mode='markers',
                marker=dict(
                    symbol='triangle-up',
                    size=12,
                    color='green',
                    line=dict(width=2)
                ),
                name='Long Entry',
                text=[f"Long Entry<br>Date: {t['date'].strftime('%Y-%m-%d')}<br>Price: {t['price']:.2f}<br>Profit: {t['profit']:.2f}%"
                      for t in long_entries],
                hoverinfo='text'
            ),
            row=1, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=[t['date'] for t in long_exits],
                y=[t['price'] * 1.01 for t in long_exits],
                mode='markers',
                marker=dict(
                    symbol='triangle-down',
                    size=12,
                    color='blue',
                    line=dict(width=2)
                ),
                name='Long Exit',
                text=[f"Long Exit<br>Date: {t['date'].strftime('%Y-%m-%d')}<br>Price: {t['price']:.2f}<br>Profit: {t['profit']:.2f}%"
                      for t in long_exits],
                hoverinfo='text'
            ),
            row=1, col=1
        )

    # Отображение сделок Short
    if short_entries:
        fig.add_trace(
            go.Scatter(
                x=[t['date'] for t in short_entries],
                y=[t['price'] * 1.01 for t in short_entries],
                mode='markers',
                marker=dict(
                    symbol='triangle-down',
                    size=12,
                    color='red',
                    line=dict(width=2)
                ),
                name='Short Entry',
                text=[f"Short Entry<br>Date: {t['date'].strftime('%Y-%m-%d')}<br>Price: {t['price']:.2f}<br>Profit: {t['profit']:.2f}%"
                      for t in short_entries],
                hoverinfo='text'
            ),
            row=1, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=[t['date'] for t in short_exits],
                y=[t['price'] * 0.99 for t in short_exits],
                mode='markers',
                marker=dict(
                    symbol='triangle-up',
                    size=12,
                    color='orange',
                    line=dict(width=2)
                ),
                name='Short Exit',
                text=[f"Short Exit<br>Date: {t['date'].strftime('%Y-%m-%d')}<br>Price: {t['price']:.2f}<br>Profit: {t['profit']:.2f}%"
                      for t in short_exits],
                hoverinfo='text'
            ),
            row=1, col=1
        )

    # Equity curve
    if isinstance(equity_curve, list):
        equity_curve = np.array(equity_curve)

    # Проверка соответствия equity_curve длине набора данных
    if len(equity_curve) > len(data):
        equity_curve = equity_curve[-len(data):]
    elif len(equity_curve) < len(data):
        padding = np.full(len(data) - len(equity_curve), equity_curve[-1])
        equity_curve = np.concatenate([equity_curve, padding])

    fig.add_trace(
        go.Scatter(
            x=data.index,
            y=equity_curve,
            name='Equity',
            line=dict(color='blue', width=2),
            text=[f"Equity: {eq:.2f}" for eq in equity_curve],
            hoverinfo='text+x'
        ),
        row=2, col=1
    )

    # Настройки графика
    fig.update_layout(
        title=dict(
            text=title,  # Заголовок передается только здесь
            x=0.5,
            xanchor='center'
        ),
        showlegend=True,
        height=800,
        template='plotly_white',
        hovermode='x unified',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )

    # Настройки осей графика
    fig.update_xaxes(
        title_text=" ",
        rangeslider_visible=False,
        gridcolor='lightgrey',
        showgrid=True
    )

    fig.update_yaxes(
        title_text="Price",
        gridcolor='lightgrey',
        showgrid=True,
        row=1, col=1
    )

    fig.update_yaxes(
        title_text="Equity",
        gridcolor='lightgrey',
        showgrid=True,
        row=2, col=1
    )

    return fig

In [61]:
# расчет и визуализация roc auc
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
import numpy as np
def is_float_in_range(lower_bound, value, upper_bound):
    return lower_bound <= value <= upper_bound

def rocauc(model, X_test, y_test):
  y_pred_proba = model.predict_proba(X_test)[:, 1]
  auc_score = roc_auc_score(y_test, y_pred_proba)
  print(f"ROC AUC Score: {auc_score:.4f}")
  fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

  if is_float_in_range(0.9, auc_score, 1.0) :
    print(f"Модель высокой надежности, пригодная для критически важных задач")
  elif is_float_in_range(0.8, auc_score, 0.9):
    print(f"Подходит для большинства бизнес-задач")
  elif is_float_in_range(0.7, auc_score, 0.8):
    print(f"Пригодна для некритичных задач; требует дополнительной проверки")
  elif is_float_in_range(0.6, auc_score, 0.7):
    print(f"Требует значительного улучшения")
  elif is_float_in_range(0.5, auc_score, 0.6):
    print(f"Немного лучше случайного угадывания")
  elif is_float_in_range(0.0, auc_score, 0.5):
    print(f"Эквивалентна случайному угадыванию")

  return auc_score
  #plt.figure(figsize=(8, 6))
  #plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {auc_score:.2f})')
  #plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
  #plt.xlim([0.0, 1.0])
  #plt.ylim([0.0, 1.05])
  #plt.xlabel('False Positive Rate')
  #plt.ylabel('True Positive Rate')
  #plt.title('Receiver Operating Characteristic (ROC) Curve')
  #plt.legend(loc="lower right")
  #plt.grid(True)
  #plt.show()


In [62]:
# Функция визуализации результатов лучшей стратегии с сигналами
def plot_best_strategy_signals(data, strategy, signals, strategy_results, title1):
    """Функция визуализации результатов лучшей стратегии с отдельными подграфиками для каждого индикатора"""
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.droplevel(1)

    # Определяем количество подграфиков в зависимости от стратегии
    if isinstance(strategy, Strategy1):
        # Для Strategy1: EMA, RSI, MACD
        num_subplots = 4  # Основной график + 3 индикатора
        subplot_titles = ["Price и сигналы", "EMA", "RSI", "MACD"]  # Убираем дублирование заголовка
    elif isinstance(strategy, Strategy2):
        # Для Strategy2: Bollinger Bands, Stochastic + RSI
        num_subplots = 4  # Основной график + 3 индикатора
        subplot_titles = ["Price и сигналы", "Bollinger Bands", "Stochastic %K", "Stochastic %D", "RSI"]  # Убираем дублирование заголовка
    elif isinstance(strategy, Strategy3):
        # Для Strategy3: RSI
        num_subplots = 2  # Основной график + 1 индикатор
        subplot_titles = ["Price и сигналы", "RSI"]  # Убираем дублирование заголовка
    elif isinstance(strategy, Strategy4):
        # Для Strategy4: RSI
        num_subplots = 5  # Основной график + 5 индикаторов
        subplot_titles = ["Price и сигналы", "RSI", "EMA", "MACD", "Bollinger Bands"]  # Убираем дублирование заголовка
    else:
        num_subplots = 2  # По умолчанию: основной график + 1 индикатор
        subplot_titles = ["Price и сигналы", "Indicator"]  # Убираем дублирование заголовка

    # Создание subplot с несколькими графиками
    fig = make_subplots(rows=num_subplots, cols=1,
                        shared_xaxes=True,
                        vertical_spacing=0.05,
                        row_heights=[0.6] + [0.1] * (num_subplots - 1),  # Основной график занимает больше места
                        subplot_titles=subplot_titles)  # Заголовки для каждого подграфика

    # Добавление свечного графика на первый подграфик
    fig.add_trace(
        go.Candlestick(
            x=data.index,
            open=data['Open'],
            high=data['High'],
            low=data['Low'],
            close=data['Close'],
            name='Цена'
        ),
        row=1, col=1
    )

    # Добавление сигналов на первый подграфик
    buy_signals = signals[signals == 1].index
    sell_signals = signals[signals == -1].index

    # Покупки
    if len(buy_signals) > 0:
        fig.add_trace(
            go.Scatter(
                x=buy_signals,
                y=data.loc[buy_signals, 'Low'] * 0.99,
                mode='markers',
                marker=dict(
                    symbol='triangle-up',
                    size=12,
                    color='green',
                    line=dict(width=2)
                ),
                name='Сигнал на покупку',
                text=[f"Покупка<br>Дата: {date.strftime('%Y-%m-%d')}<br>Цена: {data.loc[date, 'Close']:.2f}"
                      for date in buy_signals],
                hoverinfo='text'
            ),
            row=1, col=1
        )

    # Продажи
    if len(sell_signals) > 0:
        fig.add_trace(
            go.Scatter(
                x=sell_signals,
                y=data.loc[sell_signals, 'High'] * 1.01,
                mode='markers',
                marker=dict(
                    symbol='triangle-down',
                    size=12,
                    color='red',
                    line=dict(width=2)
                ),
                name='Сигнал на продажу',
                text=[f"Продажа<br>Дата: {date.strftime('%Y-%m-%d')}<br>Цена: {data.loc[date, 'Close']:.2f}"
                      for date in sell_signals],
                hoverinfo='text'
            ),
            row=1, col=1
        )

    # Добавление индикаторов на отдельные подграфики
    if isinstance(strategy, Strategy1):
        # Для Strategy1: EMA, RSI, MACD
        close_prices = data['Close'].values.astype(float).flatten()
        df = data.copy()
        df['EMA'] = ta.EMA(close_prices, timeperiod=strategy.ema_period)
        df['RSI'] = ta.RSI(close_prices, timeperiod=strategy.rsi_period)
        macd, signal, _ = ta.MACD(close_prices,
                                  fastperiod=strategy.macd_fast,
                                  slowperiod=strategy.macd_slow,
                                  signalperiod=strategy.macd_signal)
        df['MACD'] = macd
        df['Signal'] = signal

        # EMA на втором подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['EMA'],
                name='EMA',
                line=dict(color='blue', width=1)
            ),
            row=2, col=1
        )

        # RSI на третьем подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['RSI'],
                name='RSI',
                line=dict(color='orange', width=1)
            ),
            row=3, col=1
        )

        # MACD и Signal на четвертом подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['MACD'],
                name='MACD',
                line=dict(color='green', width=1)
            ),
            row=4, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['Signal'],
                name='MACD Signal',
                line=dict(color='red', width=1)
            ),
            row=4, col=1
        )

    elif isinstance(strategy, Strategy2):
        # Для Strategy2: Bollinger Bands, Stochastic + RSI
        close_prices = data['Close'].values.astype(float).flatten()
        high_prices = data['High'].values.astype(float).flatten()
        low_prices = data['Low'].values.astype(float).flatten()
        df = data.copy()
        df['BB_middle'] = ta.SMA(close_prices, timeperiod=strategy.bb_period)
        std_dev = ta.STDDEV(close_prices, timeperiod=strategy.bb_period)
        df['RSI'] = ta.RSI(close_prices, timeperiod=strategy.rsi_period)
        df['BB_upper'] = df['BB_middle'] + strategy.bb_std * std_dev
        df['BB_lower'] = df['BB_middle'] - strategy.bb_std * std_dev
        slowk, slowd = ta.STOCH(high_prices,
                               low_prices,
                               close_prices,
                               fastk_period=strategy.stoch_k,
                               slowk_period=strategy.stoch_d,
                               slowd_period=strategy.stoch_slow)
        df['slowk'] = slowk
        df['slowd'] = slowd

        # Bollinger Bands на втором подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['BB_upper'],
                name='BB Upper',
                line=dict(color='blue', width=1)
            ),
            row=2, col=1
        )

        # RSI на третьем подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['RSI'],
                name='RSI',
                line=dict(color='orange', width=1)
            ),
            row=3, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['BB_middle'],
                name='BB Middle',
                line=dict(color='green', width=1)
            ),
            row=2, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['BB_lower'],
                name='BB Lower',
                line=dict(color='red', width=1)
            ),
            row=2, col=1
        )

        # Stochastic на третьем подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['slowk'],
                name='Stochastic %K',
                line=dict(color='purple', width=1)
            ),
            row=3, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['slowd'],
                name='Stochastic %D',
                line=dict(color='orange', width=1)
            ),
            row=3, col=1
        )


    elif isinstance(strategy, Strategy3):
        # Для Strategy3: RSI
        close_prices = data['Close'].values.astype(float).flatten()
        df = data.copy()
        df['RSI'] = ta.RSI(close_prices, timeperiod=strategy.rsi_period)

        # RSI на втором подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['RSI'],
                name='RSI',
                line=dict(color='blue', width=1)
            ),
            row=2, col=1
        )
    elif isinstance(strategy, Strategy4):
        # Для Strategy4
        close_prices = data['Close'].values.astype(float).flatten()
        df = data.copy()
        df['RSI'] = ta.RSI(close_prices, timeperiod=strategy.rsi_period)
        df['BB_middle'] = ta.SMA(close_prices, timeperiod=strategy.bb_period)
        std_dev = ta.STDDEV(close_prices, timeperiod=strategy.bb_period)
        df['BB_upper'] = df['BB_middle'] + strategy.bb_std * std_dev
        df['BB_lower'] = df['BB_middle'] - strategy.bb_std * std_dev
        df['EMA'] = ta.EMA(close_prices, timeperiod=strategy.ema_period)
        macd, signal, _ = ta.MACD(close_prices,
                                  fastperiod=strategy.macd_fast,
                                  slowperiod=strategy.macd_slow,
                                  signalperiod=strategy.macd_signal)
        df['MACD'] = macd
        df['Signal'] = signal


        # RSI на втором подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['RSI'],
                name='RSI',
                line=dict(color='blue', width=1)
            ),
            row=2, col=1
        )

        # Bollinger Bands на пятом подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['BB_upper'],
                name='BB Upper',
                line=dict(color='blue', width=1)
            ),
            row=5, col=1
        )

        # EMA на втором подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['EMA'],
                name='EMA',
                line=dict(color='blue', width=1)
            ),
            row=3, col=1
        )

        # MACD и Signal на четвертом подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['MACD'],
                name='MACD',
                line=dict(color='green', width=1)
            ),
            row=4, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['Signal'],
                name='MACD Signal',
                line=dict(color='red', width=1)
            ),
            row=4, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['BB_middle'],
                name='BB Middle',
                line=dict(color='green', width=1)
            ),
            row=5, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['BB_lower'],
                name='BB Lower',
                line=dict(color='red', width=1)
            ),
            row=5, col=1
        )

    # Настройка макета
    fig.update_layout(
        title=dict(
            text=title1,  # Заголовок передается только здесь
            x=0.5,
            xanchor='center'
        ),
        showlegend=True,
        height=200 * num_subplots,  # Высота графика зависит от количества подграфиков
        template='plotly_white',
        hovermode='x unified',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )

    # Настройка осей
    fig.update_xaxes(
        title_text=" ",
        rangeslider_visible=False,
        gridcolor='lightgrey',
        showgrid=True
    )

    fig.update_yaxes(
        title_text=" ",
        gridcolor='lightgrey',
        showgrid=True,
        row=1, col=1
    )

    # Настройка осей для индикаторов
    for i in range(2, num_subplots + 1):
        fig.update_yaxes(
            title_text=" ",
            gridcolor='lightgrey',
            showgrid=True,
            row=i, col=1
        )

    return fig

In [63]:
# Функция для бэктестирования стратегии
def backtest_strategy(strategy, data):
    """Бэктестирование стратегии"""
    try:
        if data is None or len(data) < 2:
            return None

        signals, conditions, df = strategy.generate_signals(data)
        if signals is None or signals.empty:
            return None


        signals = signals.fillna(0)
        signals = signals.reindex(data.index, fill_value=0)

        close_prices = data['Close'].values
        equity = 1.0
        equity_curve = [equity]
        positions = []
        position_open = False
        entry_price = 0
        entry_signal = 0
        entry_date = None

        for i in range(1, len(signals)):
            current_signal = signals.iloc[i]
            current_date = signals.index[i]

            if not position_open and current_signal != 0:
                # Открываем позицию
                position_open = True
                entry_price = close_prices[i]
                entry_signal = current_signal
                entry_date = current_date

            elif position_open and ((current_signal != 0 and current_signal != entry_signal) or i == len(signals)-1):
                # Закрываем позицию
                exit_price = close_prices[i]
                profit = (exit_price - entry_price) / entry_price * entry_signal

                positions.append({
                    'entry_date': entry_date,
                    'exit_date': current_date,
                    'profit': float(profit)
                })

                equity *= (1.0 + profit)
                position_open = False

            equity_curve.append(float(equity))

        strategy.positions = positions
        strategy.equity = equity_curve
        strategy.signals = signals  # Сохраняем сигналы для последующей визуализации

        return strategy.calculate_metrics(data), conditions, signals, df

    except Exception as e:
        print(f"Ошибка при бэктестировании: {str(e)}")
        return None

In [64]:
def find_best_strategy(all_results):
    """ Поиск лучшей стратегии среди всех инструментов и наборов данных на основе доходности с поправкой на риск. """
    best_strategy = {
        'strategy_name': None,
        'instrument': None,
        'dataset': None,
        'metrics': None,
        'risk_adjusted_returns': float('-inf'),
        'params': None
    }

    for instrument, results in all_results.items():
        #for strategy_name in ['Strategy1', 'Strategy2', 'Strategy3', 'Strategy4']:
        for strategy_name in ['Strategy4']:
            if strategy_name not in results:
                continue

            # Check all datasets (train_default, test, valid)
            for dataset in ['train_default', 'test', 'valid', 'test_ml']:
                if dataset not in results[strategy_name]:
                    continue

                metrics = results[strategy_name][dataset]
                risk_adjusted_returns = calculate_risk_adjusted_returns(metrics)

                if risk_adjusted_returns > best_strategy['risk_adjusted_returns']:
                    params_key = 'train_default_params' if dataset == 'train_default' else 'optimized_params'
                    if dataset == 'valid':
                        params_key = 'optimized_params_combined'

                    best_strategy = {
                        'strategy_name': strategy_name,
                        'instrument': instrument,
                        'dataset': dataset,
                        'metrics': metrics,
                        'risk_adjusted_returns': risk_adjusted_returns,
                        'params': results[strategy_name].get(params_key)
                    }

    return best_strategy

In [65]:
def analyze_instrument(symbol):
    """Анализ торговых стратегий для одного инструмента"""

    try:
        data = load_data(symbol)
        if data is None or len(data) < 100:
            print(f"Недостаточно данных для {symbol}")
            return None, None

        data = data.copy()
        data = data.ffill().copy()
        #data = data.fillna(method='ffill').fillna(method='bfill')

        if data.isnull().any().any():
            print(f"Обнаружены пропущенные значения после обработки в {symbol}")
            return None, None

        print(f"\nРазмер данных: {len(data)} строк")

        train_data, test_data, valid_data = split_data(data)

        print("\nПериоды наборов данных:")
        print(f"Train: {train_data.index[0].date()} - {train_data.index[-1].date()}")
        print(f"Test: {test_data.index[0].date()} - {test_data.index[-1].date()}")
        print(f"Valid: {valid_data.index[0].date()} - {valid_data.index[-1].date()}")
        print(f"TestMl: {test_data.index[0].date()} - {test_data.index[-1].date()}")


        results = {}

        try:
            bh_results = backtest_buy_hold(data)
            if bh_results:
                results['Buy & Hold'] = {
                    'full': bh_results,
                    'full_data': data  # Сохраняем данные для расчета количества дней
                }
        except Exception as e:
            print(f"Ошибка при расчете Buy & Hold для {symbol}: {str(e)}")

        for strategy_name, strategy_params in [
            #('Strategy1', Strategy1()),
            #('Strategy2', Strategy2()),
            #('Strategy3', Strategy3()),
            ('Strategy4', Strategy4())
        ]:
            try:
                strategy_results = {}

                # Бэктестирование на Train с гиперпараметрами по умолчанию
                print(f"\n1. Набор данных Train. Начало бэктестирования {strategy_name} с гиперпараметрами по умолчанию: {strategy_params.get_params()}")
                #print(f"Гиперпараметры по умолчанию: {strategy_params.get_params()}")
                train_metrics, conditions, signals, df = backtest_strategy(strategy_params, train_data)
                if train_metrics:
                    strategy_results['train_default'] = train_metrics
                    strategy_results['train_default_params'] = strategy_params.get_params()  # Гиперпараметры
                    strategy_results['train_default_data'] = train_data  # Данные для расчета количества дней

                # Оптимизация гиперпараметров на Train
                print(f"\nНачало оптимизации гиперпараметров для {strategy_name} на Train...")
                param_space = param_spaces[strategy_name]
                best_params = optimize_strategy(globals()[strategy_name], train_data, param_space)
                if best_params:
                    print(f"Оптимальные гиперпараметры для {strategy_name} на Train: {best_params}")
                    optimized_strategy = globals()[strategy_name](**best_params)
                    strategy_results['optimized_params'] = best_params  # Оптимизированные гиперпараметры

                    # Бэктестирование на Test с оптимизированными гиперпараметрами
                    print(f"\n2. Набор данных Test. Начало бэктестирования {strategy_name} с оптимизированными гиперпараметрами на наборе Train: {best_params}")
                    #print(f"Оптимизированные гиперпараметры: {best_params}")
                    test_metrics, conditions_test, signals_test, df_test = backtest_strategy(optimized_strategy, test_data)
                    if test_metrics:
                        strategy_results['test'] = test_metrics
                        strategy_results['test_data'] = test_data  # Данные для расчета количества дней

                    # Объединение Train и Test для оптимизации
                    train_test_data = pd.concat([train_data, test_data])
                    print(f"\nОбъединенный набор данных Train и Test: {train_test_data.index[0].date()} - {train_test_data.index[-1].date()}")

                    # Оптимизация гиперпараметров на объединенном наборе
                    print(f"\nНачало оптимизации гиперпараметров для {strategy_name} на объединенном наборе Train и Test...")
                    best_params_combined = optimize_strategy(globals()[strategy_name], train_test_data, param_space)
                    print(f"Оптимальные гиперпараметры для {strategy_name} на объединенном наборе Train и Test: {best_params_combined}")
                    optimized_strategy_combined = globals()[strategy_name](**best_params_combined)
                    strategy_results['optimized_params_combined'] = best_params_combined  # Оптимизированные гиперпараметры

                    # проверка стратегии с испльзованием ML
                    print(f"\n3. Набор данных Test.  Начало бэктестирования {strategy_name} с оптимизированными гиперпараметрами на наборе Test: {best_params_combined} с использованием ML")
                    ml_metrics = backtestWithML(optimized_strategy, data, test_data)
                    #display(ml_metrics)
                    if ml_metrics:
                        strategy_results['test_ml'] = ml_metrics.copy()
                        strategy_results['test_ml_data'] = test_data.copy()  # Данные для расчета количества дней

                    # Бэктестирование на Valid с оптимизированными гиперпараметрами
                    print(f"\n4. Набор данных Valid.  Начало бэктестирования {strategy_name} с оптимизированными гиперпараметрами на объединенном наборе Train и Test: {best_params_combined}")
                    #print(f"Оптимизированные гиперпараметры: {best_params_combined}")
                    #valid_metrics, conditions, signals, df = backtest_strategy(optimized_strategy_combined, valid_data)
                    valid_metrics = backtestWithML(optimized_strategy, data, valid_data, ml_modelname=ml_metrics['ml_modelname'])
                    if valid_metrics:
                        strategy_results['valid'] = valid_metrics
                        strategy_results['valid_data'] = valid_data  # Данные для расчета количества дней
                    print("=" * 100)  # Сплошная горизонтальная линия
                if strategy_results:
                    results[strategy_name] = strategy_results

            except Exception as e:
                print(f"Ошибка при анализе стратегии {strategy_name}: {str(e)}")
                continue

        if not results:
            print(f"Не удалось получить результаты для {symbol}")
            return None, None


        return results, data

    except Exception as e:
        print(f"Ошибка при анализе инструмента {symbol}: {str(e)}")
        return None, None


# Функция отображения графика датафрейма
def display_graf(df):
  sns.set(style="darkgrid")
  plt.figure(figsize=(15, 6))
  #display(df)
  dfgr = df
  #dfgr['short_mavg'] = dfgr['Close'].rolling(50).mean()
  #dfgr['long_mavg'] = dfgr['Close'].rolling(200).mean()
  subset = dfgr[dfgr['Ticker'] == 'GOOG']
  plot_data = subset[-500:]
  plt.plot(plot_data.index, plot_data['Close'], label='GOOG')
  #plt.plot(plot_data['short_mavg'], label='50-дневная скользящая средняя')
  #plt.plot(plot_data['long_mavg'], label='200-дневная скользящая средняя')
  plt.title(' ')
  plt.xlabel('Date')
  plt.ylabel('Close Price')
  plt.legend()
  plt.show()


# Функция загрузки данных
def load_data(symbol, start_date='2022-01-01', end_date='2025-10-01'):
    """Загрузка данных с Yahoo Finance"""
    display(f"\nЗагрузка данных с Yahoo Finance {symbol}")
    df = yf.download(symbol, start=start_date, end=end_date, interval='1d', auto_adjust=True)#, interval='1h') progress=False, multi_level_index=False)

    df = df[['Open', 'High', 'Low', 'Close', 'Volume']]

    df.index = pd.to_datetime(df.index, format = '%Y-%m-%d')
    df = df.dropna()  # Удаление строк с пропущенными значениями
    df = df[~df.index.duplicated()]  # Удаление дубликатов
    #df = df.reindex(pd.date_range(start=df.index.min(),end=df.index.max(),freq='1D'))
    #df = df.interpolate(method='linear').ffill().copy()
    #df = df.interpolate(method='time').copy()


    # Перевод всех значений в числовой формат
    for c in [x for x in df.columns if x != 'Date']:
        df[c] = df[c].apply(pd.to_numeric, errors='coerce')
    # Сбрасываем мультииндекс, если он есть
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)

    #display(df)
    #fig = go.Figure()
    #fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Close'))
    #fig.update_layout(title='Визуализация индикатора close после интерполяции', xaxis_title='Дата', yaxis_title='Значение')
    #fig.show()

    return df


# Функция разделения данных на наборы
def split_data(df):
    """Разделение данных на наборы данных: Train, Test и Valid"""
    train_size = 0.6
    test_size = 0.2
    # Первое разделение на train и остаток
    train_data, temp_data = train_test_split(df, train_size=train_size, shuffle=False)
    # Разделение остатка на test и valid
    test_size_adjusted = test_size / (1 - train_size)
    test_data, valid_data = train_test_split(temp_data, train_size=test_size_adjusted, shuffle=False)
    #train_data = train_data.reindex(pd.date_range(start=train_data.index.min(),end=train_data.index.max(),freq='1D'))
    #train_data = train_data.interpolate(method='linear').ffill().copy()

    return train_data, test_data, valid_data



def main():
    """Основная функция с выбором стратегии и визуализацией"""
    all_results = {}
    # Определение списка инструментов
    #symbol = ['GOOG']

    #data = load_data(symbol)

    #if data is None or len(data) < 100:
     #       print(f"Недостаточно данных для {symbol}")
     #       return None, None

    #stock_data = data.copy()
    #stock_data = stock_data.ffill().bfill()

    #if stock_data.isnull().any().any():
    #        print(f"Обнаружены пропущенные значения после обработки в {symbol}")
    #        return None, None

    # Разделение данных на наборы
    #train_data, test_data, valid_data = split_data(stock_data)

    #display(stock_data.head(3))

    #print("\nПериоды наборов данных:")
    #print(f"Train: {train_data.index[0].date()} - {train_data.index[-1].date()}")
    #print(f"Test: {test_data.index[0].date()} - {test_data.index[-1].date()}")
    #print(f"Valid: {valid_data.index[0].date()} - {valid_data.index[-1].date()}")

    #display_graf(test_data)

    #stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'BRK-B', 'NVDA', 'META', 'JPM', 'V', '^GSPC']
    #cryptos = ['SOL-USD', 'ETH-USD', 'BTC-USD', 'XRP-USD', 'USDT-USD']
    stocks = ['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'BTC-USD']
    #cryptos = ['SOL-USD', 'BTC-USD']

    instruments = stocks
    for instrument in instruments:

        print(f"\n\033[1;34mАнализ инструмента: {instrument}\033[0m")
        print("=" * 100)  # Сплошная горизонтальная линия
        try:
            results, data = analyze_instrument(instrument)
            if results is not None and data is not None:
                all_results[instrument] = results
                summary_table = create_summary_table(results, instrument)
                display(summary_table)

        except Exception as e:
            print(f"Ошибка в анализе инструмента {instrument}: {str(e)}")
            continue

    # Поиск лучшей стратегии, используя метод отбора
    best_strategy = find_best_strategy(all_results)

    if best_strategy['strategy_name']:
        print(f"\n\033[1;34mОпределение стратегии с лучшими показателями инструменту и по набору данных\033[0m")
        print(f"Лучшая стратегия: {best_strategy['strategy_name']}")
        print(f"Лучший инструмент: {best_strategy['instrument']}")
        print(f"Лучший набор данных: {best_strategy['dataset']}")
        print(f"Доходность с корректировкой риска (Risk-Adjusted Returns): {best_strategy['risk_adjusted_returns']:.2f}")
        print("\nОптимальные гиперпараметры:")
        for param, value in best_strategy['params'].items():
            print(f"{param}: {value}")

        # Вывод метрик производительности
        metrics = best_strategy['metrics']
        print("\nМетрики производительности для лучшей стратегии:")
        print(f"Конечный капитал (final_equity) : {metrics['final_equity']:.2f}")
        print(f"Количество сделок (n_trades): {metrics['n_trades']}")
        print(f"Доходность (returns): {metrics['returns']:.2f}%")
        print(f"Максимальная просадка (max_drawdown): {metrics['max_drawdown']:.2f}%")
        print(f"Процент прибыльных сделок (win_rate): {metrics['win_rate']:.2f}%")
        print(f"Коэффициент Шарпа (sharpe_ratio): {metrics['sharpe_ratio']:.2f}")
        print(f"Коэффициент Сортино (sortino_ratio): {metrics['sortino_ratio']:.2f}")
        print(f"Фактор прибыли (profit_factor): {metrics['profit_factor']:.2f}")

        # Создание объекта стратегии с оптимальными параметрами
        strategy_class = globals()[best_strategy['strategy_name']]
        best_strategy_obj = strategy_class(**best_strategy['params'])

        # Получение набора данных
        data = load_data(best_strategy['instrument'])
        train_data, test_data, valid_data = split_data(data)
        if best_strategy['dataset'] == 'train_default':
            best_data = train_data.copy()
            backtest_strategy(best_strategy_obj, best_data)
            trades = analyze_trades(best_strategy_obj, best_data, best_strategy_obj.signals)
        elif best_strategy['dataset'] == 'test':
            best_data = test_data.copy()
            backtest_strategy(best_strategy_obj, best_data)
            trades = analyze_trades(best_strategy_obj, best_data, best_strategy_obj.signals)
        elif best_strategy['dataset'] == 'test_ml':
            best_data = test_data.copy()
            test_metrics = backtestWithML(best_strategy_obj, train_data, best_data, ml_modelname=metrics['ml_modelname'])
            metrics = test_metrics
            trades = analyze_trades(best_strategy_obj, best_data, best_strategy_obj.signals)
            #backtest_strategy(best_strategy_obj, best_data)
        else:
            best_data = valid_data.copy()
            valid_metrics = backtestWithML(best_strategy_obj, train_data, best_data, ml_modelname=metrics['ml_modelname'])
            metrics = valid_metrics
            trades = analyze_trades(best_strategy_obj, best_data, best_strategy_obj.signals)
            #backtest_strategy(best_strategy_obj, best_data)
            #trades = analyze_trades(best_strategy_obj, best_data, best_strategy_obj.signals)

        # Генерация сигналов и проведение бэктестинга
        #backtest_strategy(best_strategy_obj, best_data)

        # Анализ сделок
        #trades = analyze_trades(best_strategy_obj, best_data, best_strategy_obj.signals)

        # Отображение графика cо сделками
        ##title = (
        ##    f"Лучшая стратегия со сделками: {best_strategy['strategy_name']} на {best_strategy['instrument']} ({best_strategy['dataset']})"
        ##)
        ##fig = plot_best_strategy(
        ##    best_data,
        ##    best_strategy_obj,
        ##    best_strategy_obj.signals,
        ##    best_strategy_obj.equity,
        ##    metrics,
        ##    trades,
        ##    title
        ##)
        ##fig.show()

        # Отображение статистики сделок
        ##print("\nСтатистика сделок для лучшей стратегии:")
        ##print(f"Всего сделок: {len(trades)}")
        ##profitable_trades = sum(1 for trade in trades if trade['profit'] > 0)
        ##print(f"Прибыльных сделок: {profitable_trades} ({profitable_trades/len(trades)*100:.2f}%)")
        ##print(f"Средняя прибыль на сделку: {sum(trade['profit'] for trade in trades)/len(trades):.2f}%")

        #print("\nСтатистика сделок для лучшей стратегии:")
        #print(f"Всего сделок: {len(trades)}")
        #profitable_trades = sum(1 for trade in trades if trade['profit'] > 0)
        #print(f"Прибыльных сделок: {profitable_trades} ({profitable_trades/len(trades)*100:.2f}%)")
        #print(f"Средняя прибыль на сделку: {sum(trade['profit'] for trade in trades)/len(trades):.2f}%")

        # Отображение графика с сигналами
        #title1 = f"Сигналы лучшей стратегии: {best_strategy['strategy_name']} на {best_strategy['instrument']} ({best_strategy['dataset']})"
        #fig = plot_best_strategy_signals(
        #    best_data,
        #    best_strategy_obj,
        #    best_strategy_obj.signals,
        #    metrics,
        #    title1
        #)
        #fig.show()

    else:
        print("\nНе найдена лучшая стратегия")

if __name__ == "__main__":
  #https://sky.pro/wiki/profession/indikatory-i-ih-ispolzovanie-v-torgovle-fyuchersami/
  #https://python.ivan-shamaev.ru/example-script-calculate-ta-lib-indicators/
  #https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html
  #https://alchemymarkets.com/education/indicators/on-balance-volume-obv/
  #https://stackoverflow.com/questions/52671594/calculating-stockss-on-balance-volume-obv-in-python#60093990
  #https://trading-strategies.academy/archives/1977
  #https://github.com/TA-Lib/ta-lib-python
  #https://www.tbank.ru/invest/help/educate/trading/about/stochastic/

  main()


Анализ инструмента: AAPL


'\nЗагрузка данных с Yahoo Finance AAPL'

[*********************100%***********************]  1 of 1 completed


Размер данных: 939 строк

Периоды наборов данных:
Train: 2022-01-03 - 2024-04-01
Test: 2024-04-02 - 2024-12-27
Valid: 2024-12-30 - 2025-09-30
TestMl: 2024-04-02 - 2024-12-27

1. Набор данных Train. Начало бэктестирования Strategy4 с гиперпараметрами по умолчанию: {'bb_period': 20, 'bb_std': 2.0, 'rsi_period': 14, 'sma': 10, 'ema_period': 4, 'ma_period': 10, 'macd_fast': 12, 'macd_slow': 26, 'macd_signal': 9}

Начало оптимизации гиперпараметров для Strategy4 на Train...
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 50/50 [00:02<00:00, 24.92trial/s, best loss: -1.662180427493048]
Оптимальные гиперпараметры для Strategy4 на Train: {'bb_period': np.float64(20.0), 'bb_std': np.float64(2.354388128103639), 'ema_period': np.float64(15.0), 'ma_period': np.float64(33.0), 'macd_fast': np.float64(13.0), 'macd_signal': np.float64(15.0), 'macd_slow': np.float64(29.0), 'rsi_period': np.float64(20.0), 'sma': np.float64(45.0)}

2. Набор данных Test. Начало бэктестирования Strategy4 с оптимизированными гиперпараметрами на наборе Train: {'bb_period': np.float64(20.0), 'bb_std': np.float64(2.354388128103639), 'ema_period': np.float64(15.0), 'ma_period': np.float64(33.0), 'macd_fast': np.float64(13.0), 'macd_signal': np.float64(15.0), 'macd_slow': np.float64(29.0), 'rsi_period': np.float64(20.0), 'sma': np.float64(45.0)}

Объединенный набор данных Train и Test: 2022-01-03 - 2024-12-27

Начало оптимизации гиперпараметров для Strategy4 на объединенном наборе Train и Test...
100%|██████████| 50/50 [00:

...

'Выводим график :'

'Выводим db1 и db2 :'

,db1,db2
Start,2022-02-17 00:00:00,2024-05-16 00:00:00
End,2024-03-24 00:00:00,2024-12-27 00:00:00
Duration,766 days 00:00:00,225 days 00:00:00
Exposure Time [%],98.095238,66.666667
Equity Final [$],915593.725434,1596001.837859
Equity Peak [$],2383924.84402,1602126.956359
Commissions [$],344842.483551,111911.466828
Return [%],-8.440627,59.600184
Buy & Hold Return [%],65.854636,44.354707
Return (Ann.) [%],-26.401042,1400.736843


'risk_adjusted_return_ml = -95839.54098498115'

ROC AUC Score: 0.7194
Пригодна для некритичных задач; требует дополнительной проверки
